# 层和模块
:label:`sec_model_construction`

当我们首次介绍神经网络时，
我们专注于具有单个输出的线性模型。
在这里，整个模型仅由一个神经元组成。
请注意，单个神经元
（i）接收一组输入；
（ii）生成相应的标量输出；
（iii）具有一组可以更新的参数以优化感兴趣的某个目标函数。
然后，当我们开始考虑具有多个输出的网络时，
我们利用向量化算术来描述整个神经元层。
就像单个神经元一样，
层（i）接收一组输入，
（ii）生成相应的输出，
（iii）由一组可调参数描述。
当我们通过softmax回归工作时，
单层本身就是模型。
然而，即使我们随后
引入了多层感知机（MLPs），
我们仍然可以认为模型保留了这种基本结构。

有趣的是，对于MLPs，
整个模型及其构成的各层
都共享这种结构。
整个模型接收原始输入（特征），
生成输出（预测），
并拥有参数
（来自所有构成层的组合参数）。
同样，每个单独的层接收输入
（由前一层提供）
生成输出（后续层的输入），
并拥有一组根据从后续层反向流动的信号进行更新的可调参数。


虽然你可能认为神经元、层和模型
为我们提供了足够的抽象来进行我们的工作，
但实际上我们经常发现方便地谈论比单个层大
但比整个模型小的组件。
例如，ResNet-152架构，
在计算机视觉中非常流行，
拥有数百层。
这些层由重复的*层组*模式组成。逐层实现这样的网络可能会变得乏味。
这种担忧不仅仅是假设性的——这种设计模式在实践中很常见。
上面提到的ResNet架构
赢得了2015年的ImageNet和COCO计算机视觉竞赛
在识别和检测方面 :cite:`He.Zhang.Ren.ea.2016`
并且仍然是许多视觉任务的首选架构。
类似地，在其他领域，
包括自然语言处理和语音，
将层按各种重复模式排列的架构现在也无处不在。


为了实现这些复杂的网络，
我们引入了神经网络*模块*的概念。
模块可以描述单个层，
由多个层组成的组件，
或整个模型本身！
使用模块抽象的一个好处是它们可以组合成更大的构件，
通常是递归的。这在:numref:`fig_blocks`中进行了说明。通过定义代码来按需生成任意复杂度的模块，
我们可以编写出令人惊讶的紧凑代码
同时实现复杂的神经网络。

![多个层被组合成模块，形成更大模型的重复模式。](../img/blocks.svg)
:label:`fig_blocks`


从编程的角度来看，模块由*类*表示。
它的任何子类都必须定义一个前向传播方法
将其输入转换为输出
并存储任何必要的参数。
请注意，有些模块根本不需要任何参数。
最后，模块必须具备反向传播方法，
用于计算梯度。
幸运的是，由于自动微分
（在:numref:`sec_autograd`中介绍）
提供的幕后魔法，
在定义自己的模块时，
我们只需要关心参数
和前向传播方法。

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

[**首先，我们回顾一下用于实现多层感知机的代码**]
(:numref:`sec_mlp`)。
以下代码生成一个网络，
包含一个具有256个单元和ReLU激活函数的全连接隐藏层，
接着是一个具有十个单元的全连接输出层（无激活函数）。

In [2]:
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

X = torch.rand(2, 20)
net(X).shape

torch.Size([2, 10])

在这个例子中，我们通过实例化一个`nn.Sequential`来构建我们的模型，按照它们应该被执行的顺序将层作为参数传递。简而言之，（**`nn.Sequential`定义了一种特殊的`Module`**），这是在PyTorch中表示模块的类。它维护了一个组成`Module`的有序列表。请注意，这两个全连接层都是`Linear`类的实例，而`Linear`类本身是`Module`的子类。前向传播（`forward`）方法也非常简单：它将列表中的每个模块串联起来，将每个模块的输出作为下一个模块的输入。请注意，到目前为止，我们一直通过构造`net(X)`来调用我们的模型以获得其输出。这实际上是`net.__call__(X)`的简写。

## [**一个自定义模块**]

或许理解模块工作原理最简单的方法
就是自己实现一个。
在此之前，
我们简要总结每个模块必须提供的基本功能：


1. 通过其前向传播方法接收输入数据作为参数。
1. 通过让前向传播方法返回一个值来生成输出。请注意，输出的形状可能与输入不同。例如，上面模型中的第一个全连接层接收任意维度的输入但返回256维的输出。
1. 计算其输出相对于输入的梯度，这可以通过其反向传播方法访问。通常这是自动完成的。
1. 存储并提供执行前向传播计算所需的参数。
1. 按需初始化模型参数。


在下面的代码片段中，
我们从零开始编写一个模块，
对应一个具有256个隐藏单元的单隐藏层MLP，
和一个10维的输出层。
请注意，下面的`MLP`类继承了表示模块的类。
我们将大量依赖父类的方法，
仅提供自己的构造函数（Python中的`__init__`方法）和前向传播方法。

In [3]:
class MLP(nn.Module):
    def __init__(self):
        # Call the constructor of the parent class nn.Module to perform
        # the necessary initialization
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.out = nn.LazyLinear(10)

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input X
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

让我们首先关注前向传播方法。
请注意，它以`X`作为输入，
计算应用激活函数后的隐藏表示，
并输出其对数。
在这个`MLP`实现中，
两层都是实例变量。
要明白为什么这是合理的，想象一下
实例化两个MLP，`net1`和`net2`，
并在不同的数据上训练它们。
自然而然地，我们期望它们
代表两个不同的学习模型。

我们在构造函数中[**实例化MLP的层**]
（并在每次调用前向传播方法时**调用这些层**）。
注意一些关键细节。
首先，我们自定义的`__init__`方法
通过`super().__init__()`调用了父类的`__init__`方法，
省去了重复大多数模块适用的样板代码的麻烦。
然后我们实例化了两个全连接层，
将它们分配给`self.hidden`和`self.out`。
除非我们实现一个新的层，
否则我们不必担心反向传播方法
或参数初始化。
系统会自动生成这些方法。
让我们来试试。

In [4]:
net = MLP()
net(X).shape

torch.Size([2, 10])

模块抽象的一个关键优点是其多功能性。
我们可以通过子类化模块来创建层
（例如全连接层类），
整个模型（例如上面的`MLP`类），
或者各种中等复杂度的组件。
在接下来的章节中，我们将利用这种多功能性，
例如在处理卷积神经网络时。


## [**Sequential 模块**]
:label:`subsec_model-construction-sequential`

现在我们可以更仔细地看看
`Sequential` 类是如何工作的。
回想一下，`Sequential` 被设计为
将其他模块串联在一起。
要构建我们自己的简化版 `MySequential`，
我们只需要定义两个关键方法：

1. 一种将模块逐个添加到列表中的方法。
1. 一种前向传播方法，用于将输入按添加顺序通过模块链传递。

下面的 `MySequential` 类提供了与默认 `Sequential` 类相同的功能。

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self.add_module(str(idx), module)

    def forward(self, X):
        for module in self.children():
            X = module(X)
        return X

在`__init__`方法中，我们通过调用`add_modules`方法来添加每个模块。这些模块可以在以后通过`children`方法访问。这样，系统就能知道所添加的模块，并会正确地初始化每个模块的参数。

当我们调用`MySequential`的前向传播方法时，
每个添加的模块都会按照它们被添加的顺序执行。
我们现在可以使用我们的`MySequential`类重新实现一个MLP。

In [6]:
net = MySequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))
net(X).shape

torch.Size([2, 10])

请注意，这里使用`MySequential`
与我们之前为`Sequential`类编写的代码
（如在:numref:`sec_mlp`中所述）是完全相同的。


## [**在前向传播方法中执行代码**]

`Sequential`类使模型构建变得简单，
允许我们在不需要定义自己的类的情况下
组装新的架构。
然而，并非所有架构都是简单的链式结构。
当需要更大的灵活性时，
我们将希望定义自己的模块。
例如，我们可能希望在前向传播方法中
执行Python的控制流。
此外，我们可能希望执行
任意数学运算，
而不仅仅是依赖预定义的神经网络层。

你可能已经注意到，到目前为止，
我们网络中的所有操作
都作用于网络的激活和参数。
然而，有时我们可能希望
包含一些既不是先前层的结果
也不是可更新参数的项。
我们称这些为*常数参数*。
比如说，我们希望有一个层
计算函数
$f(\mathbf{x},\mathbf{w}) = c \cdot \mathbf{w}^\top \mathbf{x}$，
其中$\mathbf{x}$是输入，$\mathbf{w}$是我们的参数，
而$c$是某个指定的常数，
它在优化过程中不会被更新。
因此，我们实现了一个`FixedHiddenMLP`类如下。

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20))
        self.linear = nn.LazyLinear(20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(X @ self.rand_weight + 1)
        # Reuse the fully connected layer. This is equivalent to sharing
        # parameters with two fully connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

在这个模型中，
我们实现了一个隐藏层，其权重
（`self.rand_weight`）在实例化时随机初始化，
之后保持不变。
这个权重不是模型参数，
因此它永远不会通过反向传播更新。
然后网络将这个“固定”层的输出
传递给一个全连接层。

需要注意的是，在返回输出之前，
我们的模型做了一些不寻常的事情。
我们运行了一个 while 循环，测试
其 $\ell_1$ 范数是否大于 $1$，
并将输出向量除以 $2$，
直到满足条件为止。
最后，我们返回了 `X` 中元素的总和。
据我们所知，没有标准的神经网络
执行此操作。
请注意，这种特定的操作可能对
任何实际任务都没有用处。
我们的目的只是向您展示如何将
任意代码集成到神经网络计算流程中。

In [8]:
net = FixedHiddenMLP()
net(X)

tensor(-0.3836, grad_fn=<SumBackward0>)

我们可以[**混合搭配各种模块组装方式。**]
在下面的例子中，我们以一些创造性的方式嵌套模块。

In [9]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.LazyLinear(64), nn.ReLU(),
                                 nn.LazyLinear(32), nn.ReLU())
        self.linear = nn.LazyLinear(16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.LazyLinear(20), FixedHiddenMLP())
chimera(X)

tensor(0.0679, grad_fn=<SumBackward0>)

## 摘要

单个层可以是模块。
许多层可以组成一个模块。
许多模块可以组成一个模块。

模块可以包含代码。
模块处理许多杂务，包括参数初始化和反向传播。
层和模块的顺序连接由`Sequential`模块处理。


## 练习

1. 如果你将`MySequential`改为在Python列表中存储模块，会出现什么问题？
1. 实现一个模块，该模块接受两个模块作为参数，例如`net1`和`net2`，并在前向传播中返回这两个网络的连接输出。这也称为*并行模块*。
1. 假设你想连接多个相同的网络实例。实现一个工厂函数，生成同一模块的多个实例，并从中构建一个更大的网络。

[讨论](https://discuss.d2l.ai/t/55)